In [20]:
import json
import csv
import ast
import re
import os

In [22]:
# read in novel_text to set of asterisks
# read in quotation info dialogue as well
# separate 

def generate_json(TEXT_FILE, CSV_FILE, OUTPUT_FILE):
    # Read the text file and split by '===='
    with open(TEXT_FILE, 'r') as f:
        raw_conversations = re.split(r'CHAPTER| {17}\* {8}\* {8}\* {8}\* {8}\*', f.read().strip())

    # Read the CSV file
    with open(CSV_FILE, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header
        dialogue_data = list(reader)


    # print(dialogue_data[0]) ['Q0', 'Was anyone hurt?', "['Was anyone hurt?']", '[[36, 52]]', 'John Beaver', "['Mrs. Beaver']", 'Implicit', 'nan', '[[]]', '[[]]', '[[]]']

    # Process conversations
    conversations = []
    current_index = 0

    raw_conv_i = 0
    all_characters = set()
    while raw_conv_i < len(raw_conversations) and current_index < len(dialogue_data):
        raw_conversation = raw_conversations[raw_conv_i]
        conversation = []
        
        characters = set()
        while current_index < len(dialogue_data):
            current_dialogue = dialogue_data[current_index]
            raw_quote_text = current_dialogue[1]
            sub_quotation_list = ast.literal_eval(current_dialogue[2])
            if sub_quotation_list[0] in raw_conversation:
                current_index += 1
                new_text = raw_quote_text.replace('\n', ' ')
                conversation.append({
                    'character': current_dialogue[4],
                    'dialogue': new_text,
                })
                characters.add(current_dialogue[4])
                all_characters.add(current_dialogue[4])
            else:
                raw_conv_i += 1
                break
        
        if conversation != []:
            conversations.append({
                "raw text": raw_conversation,
                "characters": list(characters),
                "dialogue": conversation})
    conversations.insert(0, {"All Characters": list(all_characters)})
    with open(OUTPUT_FILE, 'w') as f:
        json.dump(conversations, f, indent=4)

# TEXT_FILE = "./project-dialogism-novel-corpus/data/AHandfulOfDust/novel_text.txt"
# CSV_FILE = './project-dialogism-novel-corpus/data/AHandfulOfDust/quotation_info.csv'
# OUTPUT_FILE = 'conversations.json'

basedir = './project-dialogism-novel-corpus/data/'
for subdir in os.listdir(basedir):
    subdir_path = os.path.join(basedir, subdir)
    
    if os.path.isdir(subdir_path):
        output_file_path = "conversations_" + str(subdir) +'.json'
        textfile = os.path.join(subdir_path, "novel_text.txt")
        csvfile = os.path.join(subdir_path, "quotation_info.csv")
        generate_json(textfile, csvfile, output_file_path)

# print(f'Successfully saved {len(conversations)} conversations to {OUTPUT_FILE}')
